<a href="https://colab.research.google.com/github/jiheddachraoui/occupancy_grid_generator/blob/main/Bird_image_GT_saver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PREPARATION

In [ ]:
import os
import sys
import csv
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from sklearn.linear_model import RANSACRegressor
from scipy import stats
from scipy.ndimage import median_filter
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [ ]:
datadir = '/content/drive/MyDrive/s_data/'
scenario='ground truth/scenario7_gt/'



for item in os.listdir(os.path.join(datadir,scenario)):
    # Construct the full path to the item
    item_path = os.path.join(os.path.join(datadir,scenario), item)
    # Check if the item is a directory
    if os.path.isdir(item_path) and item != '.ipynb_checkpoints':
        # Print the path of the subdirectory
        print(item_path)



/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_3 vehicle.nissan.patrol
/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_1 vehicle.nissan.micra
/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_2 vehicle.mini.cooper_s
/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Ego_1 vehicle.audi.a2


In [ ]:
vehicles=[]
for item in os.listdir(os.path.join(datadir,scenario)):
    # Construct the full path to the item
    if os.path.isdir(item_path) and item != '.ipynb_checkpoints':
        folder_path = os.path.join(os.path.join(datadir,scenario), item)
        image_labels=folder_path+'/bird_eye/tags/'
        imag_path=folder_path+'/bird_eye/semantic/'
        save_path=folder_path+'/bird_eye_image_semantic_grid'
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        num_files = len([f for f in os.listdir(image_labels) if os.path.isfile(os.path.join(image_labels, f))])
        print('Number of files in {} :'.format(image_labels), num_files)
        vehicles.append((image_labels,num_files,save_path))

Number of files in /content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_3 vehicle.nissan.patrol/bird_eye/tags/ : 122
Number of files in /content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_1 vehicle.nissan.micra/bird_eye/tags/ : 121
Number of files in /content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_2 vehicle.mini.cooper_s/bird_eye/tags/ : 121
Number of files in /content/drive/MyDrive/s_data/ground truth/scenario7_gt/Ego_1 vehicle.audi.a2/bird_eye/tags/ : 122


In [ ]:
vehicles

[('/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_3 vehicle.nissan.patrol/bird_eye/tags/',
  122,
  '/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_3 vehicle.nissan.patrol/bird_eye_image_semantic_grid'),
 ('/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_1 vehicle.nissan.micra/bird_eye/tags/',
  121,
  '/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_1 vehicle.nissan.micra/bird_eye_image_semantic_grid'),
 ('/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_2 vehicle.mini.cooper_s/bird_eye/tags/',
  121,
  '/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_2 vehicle.mini.cooper_s/bird_eye_image_semantic_grid'),
 ('/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Ego_1 vehicle.audi.a2/bird_eye/tags/',
  122,
  '/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Ego_1 vehicle.audi.a2/bird_eye_image_semantic_grid')]

In [ ]:
ALPHA = 1
BHETA = 1*np.pi/180
RESOLUTION = 0.1
MAX_RANGE = 70
MAP_WIDTH = 100
SPHERICAL2CARTESIAN_BIAS = 6
### Some derived parameters
# OOR_MASK = A mask to filter out measurements that are out of MAX_RANGE
MAP_SIZE_X = int(MAP_WIDTH/RESOLUTION)
MAP_SIZE_Y = int(MAP_WIDTH/RESOLUTION)
xarr = np.arange(-MAP_WIDTH/2,MAP_WIDTH/2,RESOLUTION)
yarr = np.arange(-MAP_WIDTH/2,MAP_WIDTH/2,RESOLUTION)
MAP_XX, MAP_YY = np.meshgrid(xarr, -yarr)
rgrid = np.sqrt(np.add(np.square(MAP_XX),np.square(MAP_YY)))
OOR_MASK = rgrid >= MAX_RANGE

In [ ]:
REDUCED_LABEL_COLORS = np.array([
    (255, 255, 255), # None
    (128, 64, 128),  # Road RoadLines
    (80, 80, 80),    # Building,Wall, static ,Fences
    (250, 170, 30),    # Other ,GuardRail ,RailTrack ,15Bridge
    (110, 190, 160), # Pole ;TrafficLight; Static
    (107, 142, 35),  # Vegetation
    (0, 0, 142),     # Vehicle Dynamic pedestrians
    (244, 35, 232),  # Sidewalk Ground Terrain
    (70, 130, 180),   # Water

]) / 255.0 # normalize each channel [0-1] since is what Open3D uses

In [ ]:
def post_process_labels(label_array, threshold):
    processed_array = np.copy(label_array)
    rows, cols = label_array.shape

    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            if label_array[i, j] == 5:
                neighbor_labels = label_array[i-2:i+3, j-2:j+3]
                #print(neighbor_labels)
                neighbor_counts = np.sum(neighbor_labels == 7)

                if neighbor_counts > threshold:
                    #print(neighbor_counts)
                    processed_array[i, j] = 7

    return processed_array

In [ ]:
from collections import Counter
def generate_grid(labels, grid_shape):
    labels_height, labels_width = labels.shape
    grid_height, grid_width = grid_shape

    # Calculate the stride size
    stride_height = labels_height // grid_height
    stride_width = labels_width // grid_width

    # Initiation
    reduced_grid = np.zeros(grid_shape, dtype=np.uint8)

    for i in range(grid_height):
        for j in range(grid_width):
            # Calculate the range of indices for the current cell
            start_row = i * stride_height
            end_row = start_row + stride_height
            start_col = j * stride_width
            end_col = start_col + stride_width

            # Get the labels within the current cell
            cell_labels = labels[start_row:end_row, start_col:end_col].flatten()

            if len(cell_labels) > 0:
                # Count the occurrences of each label
                label_counts = Counter(cell_labels)

                # Get the label with the maximum count
                max_label = max(label_counts, key=label_counts.get)

                # Assign the maximum label to the reduced grid cell
                reduced_grid[i, j] = max_label

    a=REDUCED_LABEL_COLORS[reduced_grid.astype(int)]

    return a

In [ ]:
for item in vehicles:
  print(item)
  for  i in range(item[1]):
    labels_path=item[0]
    save_path=item[2]
    if not os.path.exists(save_path):
      os.makedirs(save_path)

    '''if  os.path.exists(save_path+'/{}.npy'.format(i)):
            print("File {} is already created. Skipping item...".format(i))

            continue'''
    if not os.path.exists(labels_path+'/{}.npy'.format(i)):
            print("File {} not found. Skipping item...".format(i))
            all_files_exist = False
            continue
    data=np.load(labels_path+'/{}.npy'.format(i))
    labels=np.array(data).astype(int)
    labels=post_process_labels(labels, 1)

    window_size = 2
    filtered_image = np.zeros_like(labels)

    filtered_image = median_filter(labels, size=window_size)
    labels=filtered_image.astype(int)
    reduced_labels=np.zeros_like(labels)
    reduced_labels[(labels == 7) | (labels == 6)] = 1
    reduced_labels[(labels == 1)| (labels == 2) | (labels == 11) | (labels == 19)] = 2
    reduced_labels[(labels == 3)| (labels == 15) | (labels == 17) | (labels == 18)] = 3
    reduced_labels[(labels == 5)| (labels == 18) | (labels == 19)] = 4
    reduced_labels[(labels == 9)] = 5
    reduced_labels[(labels == 4)| (labels == 10) | (labels == 20)] = 6
    reduced_labels[(labels == 8)| (labels == 14) | (labels == 22)] = 7
    reduced_labels[(labels == 21)] = 8
    ### Initialize OGM
    dgm_time_0 = np.zeros((MAP_SIZE_Y,MAP_SIZE_X))




    gt_grid = generate_grid(reduced_labels,dgm_time_0.shape)
    np.save(save_path+'/{}.npy'.format(i),gt_grid)
    print(save_path+'/{}.npy'.format(i))
    ## Visualize


('/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_3 vehicle.nissan.patrol/bird_eye/tags/', 122, '/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_3 vehicle.nissan.patrol/bird_eye_image_semantic_grid')
/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_3 vehicle.nissan.patrol/bird_eye_image_semantic_grid/0.npy
/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_3 vehicle.nissan.patrol/bird_eye_image_semantic_grid/1.npy
/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_3 vehicle.nissan.patrol/bird_eye_image_semantic_grid/2.npy
/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_3 vehicle.nissan.patrol/bird_eye_image_semantic_grid/3.npy
/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_3 vehicle.nissan.patrol/bird_eye_image_semantic_grid/4.npy
/content/drive/MyDrive/s_data/ground truth/scenario7_gt/Vehicle_3 vehicle.nissan.patrol/bird_eye_image_semantic_grid/5.npy
/content/drive/MyDrive/s_data/